# Creating New GitHub Repo with Jupyter Notebook

In [1]:
import tensorflow as tf

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('real_estate_db.csv', encoding='latin-1')